### Understanding Unicode

* (a) `\x00`
* (b) The printed representation is white space.
* (c) When string is evaluated, it appear as `\x00`. When the string is printed, it appear as white space.

### Unicode Encodings
* (a) UTF-16 or -32 encode all unicode into at least 2 or 4 bytes. But UTF-8 includes some unicode to a single byte. This makes the output byte sequence shorter.
* (b) `'😧'.encode('utf-8')`. We should decode by iterate over the bytes, there are some cases where a multiple bytes corresponds to one unicode string.
* (c) `'\xC0\xC0'`

### BPE Training on TinyStories

In [1]:
from cs336_basics.train_bpe import train_bpe
from cs336_basics.utils import serialize, deserialize
import cProfile

%load_ext memory_profiler

In [2]:
%%memit

txt_path = 'data/TinyStoriesV2-GPT4-train.txt'
# txt_path = 'tests/fixtures/corpus.en'
special_tokens = ['<|endoftext|>']
vocab_size = 10*(10**3)
pr = cProfile.Profile()

# Training BPE
pr.enable()
vocab, merges = train_bpe(txt_path, vocab_size, special_tokens, progress_bar=True)
pr.disable()

# print time taken in units of hours
pr.print_stats(sort='time')

2024-04-08 15:01:26,247 - INFO - Initializing pretoken frequency table
 81%|████████  | 431364807/535813418 [11:47<02:48, 620693.35it/s]

In [3]:
serialize(vocab, 'data/out/tinystories_vocab.pkl')
serialize(merges, 'data/out/tinystories_merges.pkl')

### BPE Training on OpenWebText

In [ ]:
from cs336_basics.train_bpe import train_bpe
from cs336_basics.utils import serialize, deserialize
import cProfile

%load_ext memory_profiler

In [ ]:
%%memit

txt_path = 'data/owt_train.txt'
# txt_path = 'tests/fixtures/corpus.en'
special_tokens = ['<|endoftext|>']
vocab_size = 32*(10**3)
pr = cProfile.Profile()

# Training BPE
pr.enable()
vocab, merges = train_bpe(txt_path, vocab_size, special_tokens, progress_bar=True)
pr.disable()

# print time taken in units of hours
pr.print_stats(sort='time')

2024-04-08 14:53:50,177 - INFO - Initializing pretoken frequency table
  0%|          | 824524/535813418 [00:14<19:16, 462616.42it/s]  

In [ ]:
serialize(vocab, 'data/out/tinystories_vocab.pkl')
serialize(merges, 'data/out/tinystories_merges.pkl')